Übung: Linux Namespaces - nsenter
---------------------------------

![](images/linuxns.png)

- - -

nsenter ist ein leistungsstarkes Kommandozeilen-Tool, das es ermöglicht, in die Namespaces eines laufenden Linux-Prozesses einzutreten. 

Namespaces sind eine zentrale Funktion der Linux-Kernel-Isolierung, die es ermöglicht, Systemressourcen wie Prozesslisten, Netzwerkstacks und Dateisysteme zwischen Prozessen zu trennen. 

Mit nsenter können Administratoren und Entwickler direkt in die isolierten Umgebungen eines Containers wechseln, um Debugging, Wartung oder Konfigurationsänderungen durchzuführen.

- - -

Um die Funktionalität von Linux Namespaces und nsenter zu demonstrieren, starten wir zunächst einen Container:


In [ ]:
! kubectl run webshop-nsenter --restart=Never --image=registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0

Diesen Container sehen wir im Prozesstree mit Namen `webshop` als Subprozess von `containerd`

In [ ]:
! pstree -npTA

Mittels der Prozess-Id `$(pgrep -o -x webshop)` wechseln wir mit `nsenter` in den Container und überprüfen ob dieser 
* einen eigenen Netzwerkadapter hat
* nur seinen eigenen Prozess sieht
* und sein Filesystem eine Alpine Linux Distribution beinhaltet

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --net ip addr

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --all pstree -p

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --all cat /etc/issue

Zusatzübung:
* ersetzt `--all` in den letzten zwei Befehlen durch `--net`
* Was ist anders?

Für weitere Varianten siehe Manuel von `nsenter`

In [ ]:
! man nsenter

- - -
Zur Gegenprobe schauen wir uns Netzwerkadapter und Linux Distribution der **VM** an:

In [ ]:
! ip addr

In [ ]:
! cat /etc/issue

- - -
Die gleiche Funktionalität bietet auch `exec`, jedoch werden immer alle Linux Namespaces gewechselt:

In [ ]:
! kubectl exec pod/webshop-nsenter -- ip addr
! echo "----------------------------------"
! kubectl exec pod/webshop-nsenter -- pstree -p
! echo "----------------------------------"
! kubectl exec pod/webshop-nsenter -- cat /etc/issue

- - -
Aufräumen

In [ ]:
! kubectl delete pod/webshop-nsenter